<a href="https://colab.research.google.com/github/TonyYuanMD/OCR-summer-research-2022/blob/EasyOCR/EasyOCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import easyocr
from google.colab import files
from IPython.display import Image
import pandas as pd
import os
from operator import attrgetter, itemgetter

In [1]:
!pip install easyocr
# !pip install paddleocr
# !pip install paddlepaddle
# !pip install shapely


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 70.8 MB 18 kB/s 
     |████████████████████████████████| 47.6 MB 7.1 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
uploaded = files.upload()
Image('Paralell.jpg')

In [3]:
#os.environ['KMP_DUPLICATE_LIB_OK']="TRUE"
reader=easyocr.Reader(['ch_sim','en'])
result=reader.readtext('/content/drive/MyDrive/Colab Notebooks/Paralell.jpg')
result

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:253: UserWarning: Accessing the model URLs via the internal dictionary of the module is deprecated since 0.13 and will be removed in 0.15. Please access them via the appropriate Weights Enum instead.
  "Accessing the model URLs via the internal dictionary of the module is deprecated since 0.13 and will "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


[([[2, 3], [121, 3], [121, 44], [2, 44]], 'Chapter', 0.999614424306649),
 ([[261, 0], [427, 0], [427, 41], [261, 41]], '妖怪的姓名', 0.9762993861155043),
 ([[439, 0], [569, 0], [569, 41], [439, 41]], '(如果有)', 0.9451041964827284),
 ([[1, 41], [137, 41], [137, 81], [1, 81]], '猴壬出世', 0.7158001661300659),
 ([[263, 41], [365, 41], [365, 81], [263, 81]], '牛魔壬', 0.7261606691166284),
 ([[1, 79], [133, 79], [133, 121], [1, 121]], '大闹天宫', 0.7069041728973389),
 ([[1, 119], [137, 119], [137, 161], [1, 161]], '悟空归真', 0.9821750521659851),
 ([[260, 119], [368, 119], [368, 161], [260, 161]],
  '老虎精_',
  0.9022755026817322),
 ([[389, 117], [499, 117], [499, 161], [389, 161]],
  '黑熊精 _',
  0.5364329183202297),
 ([[515, 119], [619, 119], [619, 161], [515, 161]], '野牛精', 0.9666784384885775),
 ([[1, 159], [167, 159], [167, 201], [1, 201]], '大闹黑风山', 0.9757191300417286),
 ([[262, 160], [364, 160], [364, 200], [262, 200]], '黑风怪', 0.9820740433050622),
 ([[1, 199], [199, 199], [199, 241], [1, 241]], '猪八戒做女婿', 0.92306

In [ ]:
class OCR_Result:
  def __init__(self,leftupper_corner,coordinates,text):
    OCR_Result.leftupper_corner=leftupper_corner
    OCR_Result.coordinates=coordinates
    OCR_Result.text=text
  def division_by_10(self):
    OCR_Result.leftupper_corner=np.round(np.divide(OCR_Result.leftupper_corner,10))


In [47]:
numberofrows=len(result)
result_tuple2list=[]
for x in result:
  row=[(np.round(np.divide(x[0],10))).tolist()]
  row.append(x[1::])
  result_tuple2list.append(row)

result_sorted=sorted(result_tuple2list, key=lambda x:(x[0][0][1],x[0][0][0]))

# coordinates=[x[0] for x in result]
# coordinates_int10=np.round(np.divide(coordinates,10))
# for i in range(numberofrows):
#   result[i]=coordinates_int10[i].tolist().append(result[i][1::])

In [49]:
columns=[]
numofparalines=0

for i in range(numberofrows):
  if i==0:
    columns.append([result_sorted[i][1][0]])
  # elif abs(coordinates[i][0][1]-coordinates[i-1][0][1])<=20:
  elif result_sorted[i][0][0][1]==result_sorted[i-1][0][0][1]:
    numofparalines+=1
    if result_sorted[i][0][0][0]>result_sorted[i-1][0][0][0]:
      columns[i-numofparalines].append(result_sorted[i][1][0])
  else:
    columns.append([result_sorted[i][1][0]])

df=pd.DataFrame(columns)
# df.fillna('',inplace=True)
df.to_excel('/content/drive/MyDrive/Colab Notebooks/result.xlsx',index=False)